In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
env = gym.make('FrozenLake-v0')
env = env.unwrapped

On implémente ici 3 fonctions qui nous seront utiles pour les 2 algorithmes: 
- runEpisode : évalue une policy en jouant un episode et en renvoyant son total reward.
- evaluatePolicy : évalue une policy en lançant un épisode n fois. 
- extractPolicy : déduit la policy optimale connaissant la value function. 

Les arguments sont les suivants:
- $env$ : gym environment
- $policy$ : policy utilisée
- $gamma$ : discount factor 
- $n$ : le nombre d'épisodes joués
- $v$ : value function


In [8]:
def runEpisode(env, policy, gamma = 1.0):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

def evaluatePolicy(env, policy, gamma = 1.0,  n = 100):
    scores = [
            runEpisode(env, policy, gamma = gamma)
            for _ in range(n)]
    return np.mean(scores)

def extractPolicy(v, gamma = 1.0):
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy

On implémente ici le coeur de l'algorithme "Value Iteration" basée sur la mises à jour suivantes de la fonction action-value et de la policy: 
$V_{n+1}(s) = \max_{a} \sum_{s'} \mathbb{P}_{ss'}^{a}(r_{ss'}^{a}+\gamma V_{n}(s'))$

$\pi(s) = argmax_{a} \sum_{s'} \mathbb{P}_{ss'}^{a}(r_{ss'}^{a}+\gamma V_{n}(s'))$



In [9]:
#Value iteration

def valueIteration(env, gamma = 1.0):
    v = np.zeros(env.nS)  # initialize value-function
    max_iterations = 100000
    eps = 1e-20
    for i in range(max_iterations):
        prev_v = np.copy(v)
        for s in range(env.nS):
            q_sa = [sum([p*(r + prev_v[s_]) for p, s_, r, _ in env.P[s][a]]) for a in range(env.nA)] 
            v[s] = max(q_sa)
        if (np.sum(np.fabs(prev_v - v)) <= eps):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    return v

optimalV = valueIteration(env, gamma=1.0);
policy = extractPolicy(optimalV, gamma=1.0)
policyScore = evaluatePolicy(env, policy, gamma=1.0, n=1000)
print('Policy average score = ', policyScore)

Value-iteration converged at iteration# 1373.
Policy average score =  0.808


On implémente ici l'algorithme de "policy iteration". Pour cela on utilise de deux fonctions :
- computePolicyV : évalue la value-function pour une policy donnée en itérant de la manière suivante 
$V_{n+1}(s) = \sum_{s'} \mathbb{P}_{ss'}^{a}(r_{ss'}^{a}+\gamma V_{n}(s'))$
- policyIteration : il s'agit du coeur de l'algorithme.


In [11]:
def computePolicyV(env, policy, gamma=1.0):
    v = np.zeros(env.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

def policyIteration(env, gamma = 1.0):
    policy = np.random.choice(env.nA, size=(env.nS))  # initialize a random policy
    max_iterations = 200000
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_v = computePolicyV(env, policy, gamma)
        new_policy = extractPolicy(old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy

optimalPolicy = policyIteration(env, gamma = 1.0)
scores = evaluatePolicy(env, optimalPolicy, gamma = 1.0)
print('Average scores = ', np.mean(scores))

Policy-Iteration converged at step 3.
Average scores =  0.82
